**Чистка таблицы**

1. Удалены дубликаты строк для всех таблиц.


2. Удалены пользователи без телефонных номеров (для таблицы, по которой будет составляться витрина). Потому что было принято соглашение, что в данной ситуации телефонный номер является индетификатором заказчика. Нужно заметить, что телефонный номер может использоваться несколькими пользователями, возможно, если они из одной семьи, поэтому имя мы не считаем идентификатором.


3. Оставлены только строки, где в столбце "новый статус" стоит "доставлено". Это один из признаков, что товар был выкуплен. Опять же, это только для таблицы выкупленных, для тотальной мы оставляем остальные строки.


4. Оставляем только строки, где в столбце "КоличествоПроданоКлиенту" стоит число >=1. Если нет, то товар не выкуплен. Для тотальной таблицы остальные строки всё равно не убираем, так как считаем потеницальные значения при выкупе.


5. Удаляем строки с доставкой для удобства будущих рассчётов, так как не считаем её отдельным товаром, полную выручку за неё рассчитаем отдельно.


6. Удаляем строки с отсутствующей ценой закупки, так как будут некорректные значения. (можно не удалять, затем они сами удалятся за счёт удаления маржи)


7. Уже не в функции удаляем чеки, где некорректная маржа. Выяснили, что таких чеков 1.43% от общего количества, для более точных данных и корректных рассчётов, сотрём их. 

In [43]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

data = pd.read_csv('data/all_data_sas.csv', sep=";", encoding='cp1251') # open raw data file

In [33]:
data.head() #preview table

,Дата,ДатаДоставки,НомерЗаказаНаСайте,НовыйСтатус,СуммаЗаказаНаСайте,СуммаДокумента,МетодДоставки,ФормаОплаты,Регион,Группа2,...,СуммаДоставки,НомерСтроки,КоличествоПроданоКлиенту,ДатаЗаказаНаСайте,Телефон_new,ЭлектроннаяПочта_new,Клиент,ID_SKU,ГородМагазина,МагазинЗаказа
0,01.05.2017 0:03,03.05.2017 0:00,4513479_TR,Доставлен,761,761,Магазины,Безналичная,Москва,КРУПНОГАБАРИТНЫЙ ТОВАР,...,49,2,1,30.04.2017 0:00,55574954-50515053535277,107117_ku13@mail.ru,Марина,ID10020734553,0,NaN
1,01.05.2017 10:18,03.05.2017 0:00,4513492_TR,К отгрузке,1 878,1 878,Pick point,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,180,1,1,30.04.2017 0:00,55574851-50484949535070,121117_yu21@yandex.ru,Гость,IDL00035982755,0,NaN
2,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,7,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,IDL00038110048,0,NaN
3,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,5,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,ID000so-4578957,0,NaN
4,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,4,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,IDL00016235553,0,NaN


In [34]:
'''
    "Clean table" function takes dataframe and the list "requests" with values 0 and 1. (1=yes, 0=no)
    Meaning of positions in the list:
    1 - delete dublicates
    2 - delete 0 telephone numbers
    3 - leave only rows with the status "delivered"
    4 - delete rows where less than 1 good was sold
    5 - delete "delivery" from types of goods in a column "Номенклатура"
    6 - delete rows where "Маржа" is not null
'''

def clean_table(data, requests):
    while(len(requests) < 7):
        requests.append(0)
        
    if requests[0] == 1:
        data = data[data.duplicated(keep = 'first') == False]
    
    if requests[1] == 1:
        data.drop(data.loc[data["Телефон_new"]=='0'].index, inplace=True)
        data = data[pd.notnull(data["Телефон_new"])]
    
    if requests[2] == 1:
        data.drop(data.loc[data["НовыйСтатус"]!='Доставлен'].index, inplace=True)
    
    if requests[3] == 1:
        data.drop(data.loc[data["КоличествоПроданоКлиенту"]<1].index, inplace=True)
    
    if requests[4] == 1:
        data.drop(data.loc[data["Номенклатура"]=='Доставка'].index, inplace=True)
    
    if requests[5] == 1:
        data.drop(data.loc[data["ЦенаЗакупки"]=='0' & data["ЦенаЗакупки"]==0].index, inplace=True)
        
    if requests[6] == 1:
        data = data[pd.isnull(data['Маржа'])]   
    return data

In [44]:
requests = [1,0,0,0,1,0]
data = clean_table(data, requests)

requests = [1,0,0,0,1,0]
datatemp = clean_table(data, requests)
requests = [0,0,0,0,0,0,1]
datatemp = clean_table(datatemp, requests) #temporary table with incorrect transactions

In [47]:
series = datatemp[datatemp.columns[2]] #create series

In [48]:
series = series.unique() #series of incorrect transactions

In [53]:
print(series) #example of these transactions

['4515015_TR' '4515533_TR' '4516527_TR' ... '4950869_TR' '4968388_TR'
 '5019952_TR']


In [50]:
data = data[~data["НомерЗаказаНаСайте"].isin(series)] #delete all such transactions

In [51]:
data

,Дата,ДатаДоставки,НомерЗаказаНаСайте,НовыйСтатус,СуммаЗаказаНаСайте,СуммаДокумента,МетодДоставки,ФормаОплаты,Регион,Группа2,...,СуммаДоставки,НомерСтроки,КоличествоПроданоКлиенту,ДатаЗаказаНаСайте,Телефон_new,ЭлектроннаяПочта_new,Клиент,ID_SKU,ГородМагазина,МагазинЗаказа
0,01.05.2017 0:03,03.05.2017 0:00,4513479_TR,Доставлен,761,761,Магазины,Безналичная,Москва,КРУПНОГАБАРИТНЫЙ ТОВАР,...,49,2,1,30.04.2017 0:00,55574954-50515053535277,107117_ku13@mail.ru,Марина,ID10020734553,0,NaN
1,01.05.2017 10:18,03.05.2017 0:00,4513492_TR,К отгрузке,1 878,1 878,Pick point,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,180,1,1,30.04.2017 0:00,55574851-50484949535070,121117_yu21@yandex.ru,Гость,IDL00035982755,0,NaN
2,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,7,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,IDL00038110048,0,NaN
3,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,5,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,ID000so-4578957,0,NaN
4,01.05.2017 0:03,08.05.2017 0:00,4513499_TR,Расформирован ПВЗ,1 374,1 374,Магазины,Безналичная,Москва,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,49,4,0,30.04.2017 0:00,55575054-55515752524870,guest_103117_gu28@workmail.ru,Гость,IDL00016235553,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
730553,30.06.2017 23:08,04.07.2017 0:00,5031550_TR,Расформирован ПВЗ,3 723,"3 723,01",Магазины,Безналичная,Подольск (Московская область район),ДЕТСКОЕ ПИТАНИЕ,...,49,6,0,30.06.2017 0:00,55574953-50495349574974,97117_au14@bk.ru,Андрей,ID10018195452,0,NaN
730554,30.06.2017 23:08,04.07.2017 0:00,5031550_TR,Расформирован ПВЗ,3 723,"3 723,01",Магазины,Безналичная,Подольск (Московская область район),ДЕТСКОЕ ПИТАНИЕ,...,49,5,0,30.06.2017 0:00,55574953-50495349574974,97117_au14@bk.ru,Андрей,ID10019304654,0,NaN
730555,30.06.2017 23:13,11.07.2017 0:00,5031575_TR,К отгрузке,6 690,6 690,Pick point,Безналичная,Санкт-Петербург,"ТЕКСТИЛЬ, ТРИКОТАЖ",...,370,1,1,30.06.2017 0:00,55575348-48504953555074,117117_uu16@yandex.ru,Гость,IDL00051785149,0,NaN
730556,30.06.2017 23:58,06.07.2017 0:00,5031783_TR,Доставлен,2 819,2 819,Магазины,Безналичная,Нижний Новгород,ДЕТСКОЕ ПИТАНИЕ,...,99,2,4,30.06.2017 0:00,55575450-53495456535575,121109_ym20@gmail.com,Мария,ID000sm-0652654,0,NaN


In [5]:
data.to_csv('SAS_data_clean11.csv') #save filtered dataframe